# Sales Conversion Analysis (SQL + Python)

This project evaluates whether “high-value” customer cohorts (e.g., engineers, age 30+) convert at higher rates,
and summarizes agent-level performance using SQL executed through an in-memory SQLite database.

**Key Questions**
- Do engineers convert at higher rates than the overall population?
- Does age (30+) affect conversion rate?
- Which agents have the highest sales and what are their call patterns?


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from pathlib import Path

pd.options.display.max_rows = 10

# In-memory SQLite engine
engine = create_engine("sqlite://")

# Load CSVs
customers_df = pd.read_csv("customer.csv")
agents_df = pd.read_csv("agent.csv")
calls_df = pd.read_csv("call.csv")

# Write to SQLite (consistent lowercase naming)
customers_df.to_sql("customer", engine, if_exists="replace", index=False)
agents_df.to_sql("agent", engine, if_exists="replace", index=False)
calls_df.to_sql("call", engine, if_exists="replace", index=False)

def run_query(sql: str) -> pd.DataFrame:
    """Execute a SQL query and return the result as a DataFrame."""
    with engine.connect() as conn:
        result = conn.execute(text(sql))
        return pd.DataFrame(result.fetchall(), columns=result.keys())


## Data Sanity Checks

Before analysis, confirm that the loaded data tables are complete and contain the expected number of records.


In [5]:
run_query("SELECT COUNT(*) AS customer_count FROM customer;")

,customer_count
0,1000


In [6]:
run_query("SELECT COUNT(*) AS agent_count FROM agent;")

,agent_count
0,11


In [7]:
run_query("SELECT COUNT(*) AS call_count FROM call;")


,call_count
0,9940


## Customer Segmentation: Conversion Rates

We compare conversion rates across key customer cohorts:
- Overall population
- Engineers
- Age ≥ 30
- Engineers aged 30+


In [12]:
# Overall baseline
query_overall = """
SELECT
    COUNT(*) AS total_calls,
    SUM(ca.ProductSold) AS total_sales,
    CAST(SUM(ca.ProductSold) AS FLOAT) / COUNT(*) AS conversion_rate
FROM customer AS cu
JOIN call AS ca
    ON ca.CustomerID = cu.CustomerID;
"""
run_query(query_overall)


,total_calls,total_sales,conversion_rate
0,9925,2084,0.209975


In [13]:
# Engineers only
query_engineers = """
SELECT
    COUNT(*) AS total_calls,
    SUM(ca.ProductSold) AS total_sales,
    CAST(SUM(ca.ProductSold) AS FLOAT) / COUNT(*) AS conversion_rate
FROM customer AS cu
JOIN call AS ca
    ON ca.CustomerID = cu.CustomerID
WHERE cu.Occupation LIKE '%Engineer%';
"""
run_query(query_engineers)


,total_calls,total_sales,conversion_rate
0,3619,760,0.210003


In [14]:
# Age ≥ 30
query_age30 = """
SELECT
    COUNT(*) AS total_calls,
    SUM(ca.ProductSold) AS total_sales,
    CAST(SUM(ca.ProductSold) AS FLOAT) / COUNT(*) AS conversion_rate
FROM customer AS cu
JOIN call AS ca
    ON ca.CustomerID = cu.CustomerID
WHERE cu.Age >= 30;
"""
run_query(query_age30)


,total_calls,total_sales,conversion_rate
0,3096,659,0.212855


In [15]:
# Engineers aged 30+
query_engineer_age30 = """
SELECT
    COUNT(*) AS total_calls,
    SUM(ca.ProductSold) AS total_sales,
    CAST(SUM(ca.ProductSold) AS FLOAT) / COUNT(*) AS conversion_rate
FROM customer AS cu
JOIN call AS ca
    ON ca.CustomerID = cu.CustomerID
WHERE cu.Occupation LIKE '%Engineer%'
  AND cu.Age >= 30;
"""
run_query(query_engineer_age30)


,total_calls,total_sales,conversion_rate
0,1816,376,0.207048


**Insights**
- Engineers convert at approximately the same rate as the overall population (~21%).
- Age ≥ 30 shows a small positive difference (~0.2% higher).
- Engineers aged 30+ actually perform slightly below average.
  
Conclusion: occupation and age do not meaningfully impact conversion in this dataset.


## Sales Agent Performance

We summarize performance at the agent level — total calls, duration statistics, and sales outcomes.


In [17]:
query_agent_perf = """
SELECT
    a.Name AS agent_name,
    COUNT(*) AS calls_picked_up,
    MIN(c.Duration) AS shortest_call,
    MAX(c.Duration) AS longest_call,
    AVG(c.Duration) AS avg_duration,
    SUM(c.ProductSold) AS total_sales,
    CAST(SUM(c.ProductSold) AS FLOAT) / COUNT(*) AS conversion_rate
FROM call AS c
JOIN agent AS a
    ON c.AgentID = a.AgentID
WHERE c.PickeDup = 1
  AND c.Duration >= 0
GROUP BY a.Name
ORDER BY total_sales DESC;
"""
run_query(query_agent_perf)


,agent_name,calls_picked_up,shortest_call,longest_call,avg_duration,total_sales,conversion_rate
0,Gloria Singh,662,36,349,182.175227,209,0.315710
1,Todd Morrow,630,29,339,181.003175,204,0.323810
2,Lisa Cordova,639,46,344,179.214397,201,0.314554
3,Michele Williams,685,22,306,177.880292,198,0.289051
4,Paul Nunez,647,38,323,181.358578,194,0.299845
...,...,...,...,...,...,...,...
6,Christopher Moreno,649,47,363,177.979969,189,0.291217
7,Jocelyn Parker,621,40,336,180.326892,184,0.296296
8,Dana Hardy,554,49,356,177.203971,182,0.328520
9,Randy Moore,600,16,326,178.595000,177,0.295000


**Agent Insights**
- Some agents handle significantly more calls while maintaining similar average call durations.
- Conversion rates between agents vary moderately (~26%–33%), indicating differences in effectiveness.
- Negative durations were removed to ensure valid statistics.

These summaries allow for targeted performance benchmarking rather than manually reviewing all call data.


## Conclusions

- SQL and Python were used to integrate and analyze sales call data from multiple CSVs.
- “Engineer” status does not predict higher conversion rates.
- Age 30+ may have a small effect, but it is statistically insignificant.
- Agent performance summaries identify productive patterns and potential outliers.
  
This workflow demonstrates SQL integration, business data cleaning, and exploratory analytics — suitable for business intelligence and data analysis pipelines.
